In [1]:
"""
This code is part of the DeFake video processing pipeline, detecting scene shifts in the video and cutting it accordingly.
The video is returned in a nested list format [scene][frame].

Thanks to https://github.com/Breakthrough/PySceneDetect for the scene detection framework!

@author Bryce Gernon
"""

from scenedetect import VideoManager
from scenedetect import SceneManager
from scenedetect.detectors import ContentDetector
import cv2

MOVIE_PATH = "movies/MatrixFight.mp4"

def find_scenes(video_path, threshold=25.0):
    """
    Finds all scenes in the given video and returns the frame numbers at which each scene starts and ends.
    
    :param str video_path: The relative path to the video file.
    :param float threshold=25.0: The confidence threshold of the scene detector.
    :return: A list of the frame numbers at which each scene starts and ends.
    """
    # Create our video & scene managers, then add the detector.
    video_manager = VideoManager([video_path])
    scene_manager = SceneManager()
    scene_manager.add_detector(ContentDetector(threshold=threshold))

    # Improve processing speed by downscaling before processing.
    video_manager.set_downscale_factor()

    # Start the video manager and perform the scene detection.
    video_manager.start()
    scene_manager.detect_scenes(frame_source=video_manager)

    # Each returned scene is a tuple of the (start, end) timecode.
    return scene_manager.get_scene_list()

def cut_movie(video_path, threshold=25.0):
    """
    Cuts the given movie into different scenes using a scene detector.
    
    :param str video_path: The relative path to the video file.
    :param float threshold=25.0: The confidence threshold of the scene detector.
    :return: Image data in nested list format [scene][frame].
    :raises ValueError: If the video file is not in .mp4 or .mp3 format.
    :raises AttributeError: If the path to the video is not a valid string.
    """
    try:
        if (video_path.endswith("mp4") == False) and (video_path.endswith("mp3") == False):
            raise ValueError("Video file is not in mp4/mp3 format.")
    except AttributeError:
        raise AttributeError("Path to video is of invalid type " + str(type(video_path)) + ".")
        
    scenes = find_scenes(video_path)
    video_capture = cv2.VideoCapture(video_path)
    
    success = True
    video_time = 0
    video_slices = []
    video_batch = []
    scene_num = 0
    frame_num = 0
    
    while(success == True):
        success, image = video_capture.read()
        frame_num += 1
        if (success == False):
            #print("Done cutting video!")
            return video_batch
        video_slices.append(image)
        if frame_num == scenes[scene_num][1].get_frames():
            video_batch.append(video_slices)
            video_slices = []
            scene_num += 1
            #print("Performed a cut at " + str(frame_num/video_capture.get(cv2.CAP_PROP_FPS)) + "s")
    
   
def main():
    sliced_movie = cut_movie(MOVIE_PATH)
    
if __name__ == "__main__":
    main()

100%|██████████| 8820/8820 [00:21<00:00, 419.99frames/s]


Performed a cut at 1.5333333333333334s
Performed a cut at 4.633333333333334s
Performed a cut at 6.733333333333333s
Performed a cut at 8.733333333333333s
Performed a cut at 10.5s
Performed a cut at 12.033333333333333s
Performed a cut at 12.8s
Performed a cut at 14.9s
Performed a cut at 16.4s
Performed a cut at 18.6s
Performed a cut at 20.133333333333333s
Performed a cut at 20.933333333333334s
Performed a cut at 21.9s
Performed a cut at 23.166666666666668s
Performed a cut at 23.7s
Performed a cut at 25.633333333333333s
Performed a cut at 26.733333333333334s
Performed a cut at 27.7s
Performed a cut at 29.3s
Performed a cut at 31.7s
Performed a cut at 33.13333333333333s
Performed a cut at 34.5s
Performed a cut at 35.36666666666667s
Performed a cut at 40.4s
Performed a cut at 43.766666666666666s
Performed a cut at 45.53333333333333s
Performed a cut at 48.1s
Performed a cut at 51.8s
Performed a cut at 56.53333333333333s
Performed a cut at 57.3s
Performed a cut at 58.733333333333334s
Performe